<a href="https://colab.research.google.com/github/sdsc-bw/DataFactory/blob/develop/demos/04_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning

To find a suitable model for your ML problem is very important. Not every model has the same performane on every task. Some models can be to simple (underfitting) and some models can be to complex for for a problem (overfitting). Also a model has different hyperparameters which also have an impact on the performance. Therefor exist libraries that can be used to find a appropriate model and its hyperparameters. Popular ones are [auto-sklearn](https://papers.neurips.cc/paper/2015/hash/11d0e6287202fced83f79975ec59a3a6-Abstract.html) and [hyperopt](https://github.com/hyperopt/hyperopt).

Not every model fits for every problem. In this notebook we can see the F1 scores of several models on different datasets. 

The F1 score is the harmonic mean of the precision and the recall: 
$$F1 = 2 * \frac{precision * recall}{precision + recall}$$
The higher the F1 score, the better the prediction. Precision and recall are defined as:

$$Precision = \frac{TP}{TP + FP}, Recall = \frac{TP}{TP + FN}$$
TP: True Positive, FN: False Negative, FP: False Positive

# How To use in the Datafactory

## Import packages

In [1]:
import sys

In [2]:
if 'google.colab' in sys.modules:
    !git clone https://github.com/sdsc-bw/DataFactory.git
    !ls
    
    !sudo apt-get install build-essential swig
    !curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
    !pip install auto-sklearn
    
    !pip install scipy
    
    !pip install tsai

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.utils import shuffle
from hyperopt import hp

if 'google.colab' in sys.modules:
    root = 'DataFactory/'
else:
    root = '../'
sys.path.append(root)

from datafactory.preprocessing.cleaning import clean_data
from datafactory.preprocessing.loading import split_data
from datafactory.finetuning.finetuning_hyperopt import finetune_hyperopt
from datafactory.finetuning.finetuning_auto_sklearn import finetune_auto_sklearn

2022-01-15 15:33:54,312 - numba.cuda.cudadrv.driver - init


## Load dataset: Diabetes dataset

In [5]:
dataset = load_wine()
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
df['class'] = pd.Series(dataset.target)
df = clean_data(df)
X, y, _ = split_data(df)

2022-01-15 15:33:57,000 - datafactory.util.constants - Start to clean the given dataframe...
2022-01-15 15:33:57,002 - datafactory.util.constants - Number of INF- and NAN-values are: (0, 0)
2022-01-15 15:33:57,003 - datafactory.util.constants - Set type to float32 at first && deal with INF
2022-01-15 15:33:57,004 - datafactory.util.constants - Remove columns with half of NAN-values
2022-01-15 15:33:57,006 - datafactory.util.constants - Remove constant columns
2022-01-15 15:33:57,010 - datafactory.util.constants - ...End with Data cleaning, number of INF- and NAN-values are now: (0, 0)


## How To Use DataFactory

### Hyperopt

We provided a function to use hyperopt. Some of the models require finetuning with hyperopt. 

In [6]:
# list with models to try out
models = ['decision_tree', 'random_forest', 'ada_boost', 'inception_time', 'res_net']

In [7]:
# loss in this case refers to -f1
model = finetune_hyperopt(X, y, strategy='random', models=models, cv=5, max_evals=32, mtype='C')

,Model,Score,Hyperparams,Time
0,ResNet,0.990741,"{'epochs': 25, 'lr_max': 0.001}",11.937192
1,Random Forest,0.983051,"{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}",0.430953
2,Random Forest,0.977495,"{'max_depth': 50, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}",0.185529
3,Random Forest,0.977495,"{'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 200}",0.690807
4,Random Forest,0.977401,"{'max_depth': 2, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}",0.659948
5,Random Forest,0.966196,"{'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}",0.266305
6,Random Forest,0.966196,"{'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}",0.428028
7,ResNet,0.935185,"{'epochs': 25, 'lr_max': 0.001}",11.431798
8,ResNet,0.916667,"{'epochs': 25, 'lr_max': 0.001}",11.544339
9,ResNet,0.916667,"{'epochs': 25, 'lr_max': 0.001}",11.414284


100%|███████████████████████████████████████████████| 32/32 [02:47<00:00,  5.24s/trial, best loss: -0.9907407363255819]


We can define the models that we want to test. Then we have to define a *params* variable that defines the strategy how to examine the search space. There we also can define the parameters of the search space. If parameters for models are not given, it uses our standard search space.

If we want to define custom parameters, they should be defined with the functions of hyperopt. Look at the [sklearn](https://scikit-learn.org/stable/) and [tsai](https://github.com/timeseriesAI/tsai) website to find the hyperparamters of the models. Attention: The identifier of the hyperparameters need to be unique for ever parameter (also between models).

In [8]:
models = ['decision_tree', 'random_forest', 'ada_boost', 'inception_time', 'res_net']
# attention the label has to be unique for every parameter (also between models)
decision_tree_params = {'max_depth': hp.quniform('max_depth_dt', 1, 10, 1), 
                        'criterion': hp.choice('criterion_dt', ['gini', 'entropy']), 
                        'min_samples_leaf': hp.choice('min_samples_leaf_dt', [1, 2, 4])}
random_forest_params = {'max_depth': hp.choice('max_depth_rf', [1, 2, 3, 5, 10, 20, 50]), 
                        'n_estimators': hp.choice('n_estimators_rf', [50, 100, 200])}
ada_boost_params = {'n_estimators': hp.choice('n_estimators_ab', [50, 100, 200]), 
                    'learning_rate': hp.choice('learning_rate_ab', [0.001,0.01,.1,1.0])}
inception_time_params = {'epochs': hp.choice('epochs_it', [50, 100, 150]), 
                         'lr_max': 1e-3, 
                         'opt_func':  hp.choice('optimizer_it', ['adam', 'sgd']), 
                         'loss_func': hp.choice('loss_it', ['cross_entropy', 'smooth_cross_entropy']), 
                         'batch_tfms': hp.choice('batch_tfms_it', [['standardize', 'clip', 'mag_scale'], []]), 
                         'batch_size': [64, 128], 
                         'splits': None, 
                         'metrics': ['accuracy'],
                         'nf': hp.choice('nf_it', [32, 64]), 
                         'nb_filters': hp.choice('nb_filters_it', [32, 64, 96, 128])}
res_net_params = {'epochs': hp.choice('epochs__res_net', [50, 100, 150]), 
                  'lr_max': 1e-3, 
                  'opt_func':  hp.choice('optimizer__res_net', ['adam', 'sgd']), 
                  'loss_func': hp.choice('loss__res_net', ['cross_entropy', 'smooth_cross_entropy']), 
                  'batch_tfms': hp.choice('batch_tfms_res_net', [['standardize', 'clip', 'mag_scale'], []]), 
                  'batch_size': [64, 128], 
                  'splits': None, 
                  'metrics': ['accuracy']}
# put every hyperparameter definition in an own dictionary
params = {'decision_tree': decision_tree_params, 
          'random_forest': random_forest_params, 
          'ada_boost': ada_boost_params, 
          'inception_time': inception_time_params, 
          'res_net': res_net_params}

In [9]:
# search strategy should be in ['parzen', 'random']
model = finetune_hyperopt(X, y, strategy='random', models=models, cv=5, mtype='C', params=params.copy())

,Model,Score,Hyperparams,Time
0,ResNet,1.000000,"{'batch_size': (64, 128), 'batch_tfms': (), 'epochs': 150, 'loss_func': 'smooth_cross_entropy', 'lr_max': 0.001, 'opt_func': 'sgd', 'splits': None}",68.652344
1,InceptionTime,1.000000,"{'batch_size': (64, 128), 'batch_tfms': ('standardize', 'clip', 'mag_scale'), 'epochs': 150, 'loss_func': 'cross_entropy', 'lr_max': 0.001, 'nb_filters': 64, 'nf': 32, 'opt_func': 'adam', 'splits': None}",104.737286
2,ResNet,1.000000,"{'batch_size': (64, 128), 'batch_tfms': (), 'epochs': 150, 'loss_func': 'cross_entropy', 'lr_max': 0.001, 'opt_func': 'adam', 'splits': None}",79.153059
3,InceptionTime,1.000000,"{'batch_size': (64, 128), 'batch_tfms': (), 'epochs': 100, 'loss_func': 'cross_entropy', 'lr_max': 0.001, 'nb_filters': 64, 'nf': 32, 'opt_func': 'sgd', 'splits': None}",70.050056
4,ResNet,1.000000,"{'batch_size': (64, 128), 'batch_tfms': (), 'epochs': 150, 'loss_func': 'cross_entropy', 'lr_max': 0.001, 'opt_func': 'sgd', 'splits': None}",77.701328
5,InceptionTime,1.000000,"{'batch_size': (64, 128), 'batch_tfms': (), 'epochs': 150, 'loss_func': 'cross_entropy', 'lr_max': 0.001, 'nb_filters': 64, 'nf': 64, 'opt_func': 'adam', 'splits': None}",183.858962
6,ResNet,1.000000,"{'batch_size': (64, 128), 'batch_tfms': (), 'epochs': 50, 'loss_func': 'smooth_cross_entropy', 'lr_max': 0.001, 'opt_func': 'sgd', 'splits': None}",22.298556
7,ResNet,1.000000,"{'batch_size': (64, 128), 'batch_tfms': (), 'epochs': 50, 'loss_func': 'smooth_cross_entropy', 'lr_max': 0.001, 'opt_func': 'sgd', 'splits': None}",29.456243
8,ResNet,1.000000,"{'batch_size': (64, 128), 'batch_tfms': ('standardize', 'clip', 'mag_scale'), 'epochs': 50, 'loss_func': 'smooth_cross_entropy', 'lr_max': 0.001, 'opt_func': 'adam', 'splits': None}",23.208064
9,ResNet,1.000000,"{'batch_size': (64, 128), 'batch_tfms': (), 'epochs': 100, 'loss_func': 'cross_entropy', 'lr_max': 0.001, 'opt_func': 'adam', 'splits': None}",48.454832


100%|██████████████████████████████████████████████████████████████| 32/32 [17:07<00:00, 32.11s/trial, best loss: -1.0]


### Auto-sklearn

Auto-sklearn requires a linux OS (otherwise it can be run on colab). It is an automated machine learning toolkit using sklearn models. It automatically trains different ML models with different hyperparameters. At the end it selects the best model. In the DataFactory you can use it like that:

In [ ]:
model = finetune_auto_sklearn(X, y, mtype='C')